In [1]:
import pandas as pd
import os
import tqdm

In [2]:
doorcard_2023 = pd.read_csv('./door_cards_2023.csv')[
['ARU', 'Location', 'Start Date', 'Start Time', 'End Date', 'End Time']
]
doorcard_2023.dropna(subset=['ARU'], inplace=True)

doorcard_2023["Start Datetime"] = pd.to_datetime(doorcard_2023["Start Date"] + " " + doorcard_2023["Start Time"], format="%m/%d/%y %H:%M")

doorcard_2023["End Datetime"] = pd.to_datetime(doorcard_2023["End Date"] + " " + doorcard_2023["End Time"], format="%m/%d/%y %H:%M")

In [3]:
recordings_2023 = os.listdir('../ARU_data_all/2023/')
recordings_2023 = pd.DataFrame(recordings_2023, columns=['filename'])
recordings_2023 = recordings_2023[~ recordings_2023['filename'].str.startswith('.')]

In [4]:
recordings_2023['datetime'] = pd.to_datetime(
    (recordings_2023['filename']
     .str.split('_').str[1:3].str.join('_')
     .str.split('.').str[0]),
    format='%Y%m%d_%H%M%S')

In [5]:
filenames = []
points = [] 

for _, row in tqdm.tqdm(doorcard_2023.iterrows(), total=len(doorcard_2023)):
    for _, recording in recordings_2023.iterrows():
        recorder_doorcard = row['ARU']
        recorder_recording = recording['filename'].split('_')[0]
        if recorder_doorcard != recorder_recording:
            continue
        if row['Start Datetime'] <= recording['datetime'] <= row['End Datetime']:
            filenames.append(recording['filename'])
            points.append(row['Location'])

100%|██████████| 84/84 [00:50<00:00,  1.65it/s]


In [6]:
aru2point_2023 = pd.DataFrame({'filename': filenames, 'point': points})

In [9]:
aru2point_2023.to_csv('./aru2point_2023.csv', index=False)